In [1]:
import numpy as np

In [2]:
np.set_printoptions(suppress=True, precision=2)

# Problem Description

In [3]:
def profit(production_rate,profit_per_product):
    return np.sum(production_rate*profit_per_product)

In [4]:
profit_per_product = np.array([90,60,45])

In [5]:
material_lower_bound = np.array([0., 0., 0., 0., 0.])
material_upper_bound = np.array([450., 250., 800., 450., 600.])

In [6]:
Q = np.zeros((5,3))

Q[0,:] = np.array([1.5 , 1.0 , 0.0 ])
Q[1,:] = np.array([2.0 , 0.0 , 1.5 ])
Q[2,:] = np.array([0.75, 3.0 , 0.0 ])
Q[3,:] = np.array([1.25, 0.75, 1.0 ])
Q[4,:] = np.array([1.0 , 0.0 , 2.0 ])

In [7]:
product_lower_bound = [0,0,0]

U = np.tile(material_upper_bound,3).reshape((3,-1))
U[Q.T==0] = np.nan

product_upper_bound = np.nanmin(np.divide(U,Q.T),axis=1)

In [8]:
product_upper_bound

array([125.  , 266.67, 166.67])

# Random Simulation

In [9]:
def random(lower_bound:np.ndarray, upper_bound:np.ndarray):
    return np.random.uniform(low=lower_bound, high=upper_bound)

In [10]:
random(product_lower_bound,product_upper_bound)

array([110.72, 238.86, 127.31])

In [11]:
Nsimulation = 100_000

O,I,R = [],[],[]

for i in range(Nsimulation):
    O.append(random(product_lower_bound,product_upper_bound))
    I.append(Q.dot(O[i]))
    R.append(0 if np.any(I[i]>material_upper_bound) else profit(O[i],profit_per_product))

income = np.array(R)

In [12]:
O[np.argmax(income)]

array([110.85, 238.9 ,  17.87])

In [13]:
I[np.argmax(income)]

array([405.17, 248.5 , 799.83, 335.6 , 146.59])

In [14]:
material_upper_bound-I[np.argmax(income)]

array([ 44.83,   1.5 ,   0.17, 114.4 , 453.41])

In [15]:
np.sum(material_upper_bound-I[np.argmax(income)]).tolist()

614.3169534783098

In [16]:
income[np.argmax(income)].tolist()

25114.217426613355

# Multivariate Local Optimization

In [17]:
from scipy import optimize

In [18]:
B = optimize.Bounds(product_lower_bound,product_upper_bound)
C = optimize.LinearConstraint(Q,material_lower_bound,material_upper_bound)

H = lambda x,v: np.zeros((3,3))

f = lambda x,y: -profit(x,y)
T = optimize.minimize(f,np.zeros(3),args=(profit_per_product,),method='trust-constr',constraints=C,bounds=B,hess=H)

In [19]:
T.x

array([125.  , 235.42,   0.  ])

In [20]:
Q.dot(T.x)

array([422.92, 250.  , 800.  , 332.81, 125.  ])

In [21]:
material_upper_bound-Q.dot(T.x)

array([ 27.08,   0.  ,   0.  , 117.19, 475.  ])

In [22]:
np.sum(material_upper_bound-Q.dot(T.x)).tolist()

619.2708903354736

In [23]:
profit(T.x,profit_per_product).tolist()

25374.996799996854

# Global Optimization

In [25]:
optimize.shgo(f,B,constraints=C,args=(profit_per_product,),iters=10)

 message: Optimization terminated successfully.
 success: True
     fun: -25375.00000010695
    funl: [-2.538e+04 -2.538e+04 -2.537e+04 -2.444e+04 -2.444e+04]
       x: [ 1.250e+02  2.354e+02  1.690e-10]
      xl: [[ 1.250e+02  2.354e+02  1.690e-10]
           [ 1.250e+02  2.354e+02  1.140e-10]
           ...
           [ 6.250e+01  2.510e+02  8.333e+01]
           [ 6.250e+01  2.510e+02  8.333e+01]]
     nit: 10
    nfev: 313
   nlfev: 250
   nljev: 39
   nlhev: 0